In [ ]:
#script to test the pretrained CNN models
#includes solution time, model loss, model metrics, and spectral RMSE
#2 layer systems

import warnings
warnings.filterwarnings('ignore')

import numpy as np
from numpy import round
import tensorflow as tf
from keras import callbacks
from keras.callbacks import LearningRateScheduler as LRS
from keras import models as mo
from keras.models import Model
from keras.layers import *
from keras import optimizers as opt
import keras.backend as K
import datetime
global date
date = datetime.datetime.now()
import h5py

#data rescaling fcns
def resc_mat(in_mat):
    resc = in_mat / 4
    return resc
def resc_th(in_th):
    resc = in_th* 1E7
    return resc
def resc_ang(in_theta):
    resc = in_theta / 45
    return resc
def resc_psi(in_psi):
    resc = in_psi/90
    return resc
def resc_delt(in_delt):
    resc = in_delt/90
    return resc

#import the data from file
def readin_data(filename,nmat,nang,nlay,nwave):
    f = h5py.File(filename,'r')
    arrd = np.array(f.get('data'))
    itl=0
    ith=nang
    theta = (arrd[:,itl:ith])
    itl=ith
    ith +=nmat
    ml1 = (arrd[:,itl:ith])
    itl=ith
    ith +=nmat
    ml2 = (arrd[:,itl:ith])
    itl=ith
    ith +=nlay
    th = (arrd[:,itl:ith])
    itl=ith
    ith +=nwave*nang
    rp = (arrd[:,itl:ith])
    itl=ith
    ith +=nwave*nang
    rs = (arrd[:,itl:ith])
    itl=ith
    ith +=nwave*nang
    tp = (arrd[:,itl:ith])
    itl=ith
    ith +=nwave*nang
    ts = (arrd[:,itl:ith])
    itl=ith
    ith +=nwave*nang
    psi = (arrd[:,itl:ith])
    itl=ith
    delta = (arrd[:,itl:])
    f.close()

    psi = resc_psi(psi)
    delta = resc_delt(delta)
    th = resc_th(th)
    return (ml1,ml2,th,theta,rp,rs,tp,ts,psi,delta)

#function to split the data into individual test sets
def split_data(m1,m2,th,ang,rp,rs,tp,ts,psi,delta,mn,mx):
    tr_m1=m1[mn:mx,:]
    tr_m2=m2[mn:mx,:]
    tr_th=th[mn:mx,:]
    tr_ang=ang[mn:mx,:]
    tr_p=psi[mn:mx,:]
    tr_d=delta[mn:mx,:]
    tr_rp=rp[mn:mx,:]
    tr_rs=rs[mn:mx,:]
    tr_tp=tp[mn:mx,:]
    tr_ts=ts[mn:mx,:]
    return (tr_m1,tr_m2,tr_th,tr_ang,tr_rp,tr_rs,tr_tp,tr_ts,tr_p,tr_d)


In [ ]:
#read in data from file
num_mat = 5 #number of materials probed by system
num_ang = 3 #number of angles probed by system
num_lay = 2 #number of layers in the system
num_wave = 200
#load the data file into memory
#the first position is the file name (including directory) for the data file
(ml1,ml2,th,ang,rp,rs,tp,ts,psi,delta) = readin_data('/home/arl92/Documents/newdata/data_rte_gen2lay5mat_0ge_240000n_v-tma_20201112.h5',num_mat,num_ang,num_lay,num_wave)

tr = 200000
va = tr+20000
te = va+20000

#cuts the dataset into an independent test set not used in training
#delete the loaded data not used in the test set to save memory
(m1e,m2e,the,ange,rpe,rse,tpee,tse,psie,deltae) = split_data(ml1,ml2,th,ang,rp,rs,tp,ts,psi,delta,va,te)
del ml1, th, ang, rp, rs, tp, ts, psi, delta

te_p = np.zeros((te-va,num_wave,num_ang))
te_d = np.zeros((te-va,num_wave,num_ang))
p = np.transpose(psie)
d = np.transpose(deltae)
for i in range(te-va):
    te_p[i,:,:] = np.transpose(np.reshape(p[:,i],(num_ang,num_wave)))
    te_d[i,:,:] = np.transpose(np.reshape(d[:,i],(num_ang,num_wave)))
    
te_rp = np.zeros((te-va,num_wave,num_ang))
te_rs = np.zeros((te-va,num_wave,num_ang))
p = np.transpose(rpe)
d = np.transpose(rse)
for i in range(te-va):
    te_rp[i,:,:] = np.transpose(np.reshape(p[:,i],(num_ang,num_wave)))
    te_rs[i,:,:] = np.transpose(np.reshape(d[:,i],(num_ang,num_wave)))

te_tp = np.zeros((te-va,num_wave,num_ang))
te_ts = np.zeros((te-va,num_wave,num_ang))
p = np.transpose(tpee)
d = np.transpose(tse)
for i in range(te-va):
    te_tp[i,:,:] = np.transpose(np.reshape(p[:,i],(num_ang,num_wave)))
    te_ts[i,:,:] = np.transpose(np.reshape(d[:,i],(num_ang,num_wave)))



In [ ]:
#reflectance/transmittance to structure model
#runtime results

#load the model into memory. Point this to the correct filename
model = mo.load_model('/home/arl92/Documents/newdata/rt2mt/general_2lay4matinverse_model_v-tma-allangle_convolution_RPRSTPTS_1step_20201117')
#calculate the timing results
ti = datetime.datetime.now()
results = model.predict([te_rp,te_rs,te_tp,te_ts])
tf = datetime.datetime.now()
dt = tf-ti
#print the timing results to the output. Returns the average solution time over the enrite test dataset
print('Time Ellapsed:',dt)
print('Time per call:',dt/(te-va))

In [ ]:
#reflectance/transmittance to structure model
#model metrics

#save the predicted results 
th = results[0];
l1 = np.argmax(results[1],1);
l2 = np.argmax(results[2],1);
#evaluate the model loss and metrics (accuracy in materials, thickness mse) against the test dataset
print(model.evaluate([te_rp,te_rs,te_tp,te_ts],[the,m1e,m2e]))

In [ ]:
#reflectance/transmittance to structure model
#spectral RMSE

#import materials
#the wavelengths are copied to coinside with the generated data
wave = np.linspace(450,950,200)*1E-9
ag = bb.nk_material('Ag',wave)
au = bb.nk_material('Au',wave)
al2o3 = di.nk_material('al2o3',wave)
tio2 = di.nk_material('tio2',wave)
ito = di.nk_material('ito',wave)
gl = di.nk_Cauchy_Urbach(wave,1.55,0.005)
void = np.ones(wave.size)
#define the materials array
materials = np.array([ag,al2o3,ito,au,tio2])
ang = np.array([25.,45.,65.])
n_subst = gl #all structures are simulated on a glass substrate

#how many structures to generate spectral RMSE results
numel = 2000 #should be enough to generate a good average
results = np.ones((numel,5))

#decode the materials
mats1 = np.argmax(m1e,1)
mats2 = np.argmax(m2e,1)

#calculate the spectral RMSE results
for exp in tqdm(range(numel)):
    l = th[exp]*1E-7 #data un-rescaling
    n = np.zeros((l.size,wave.size),dtype=complex)
    rp = np.zeros((wave.size,ang.size))
    rs = np.zeros((wave.size,ang.size))
    tp = np.zeros((wave.size,ang.size))
    ts = np.zeros((wave.size,ang.size))
    
    n = np.array([materials[l1[exp]],materials[l2[exp]]]) #uses the predicted material
    #calculate the spectra corrsponding to the model predicted structure
    for j in range(0,ang.size):     
        for i in range(0,wave.size):
            rp[i,j] = tmm.reflect_amp(1,ang[j], wave[i], n[:,i], l, 1., n_subst[i])
            rs[i,j] = tmm.reflect_amp(0,ang[j], wave[i], n[:,i], l, 1., n_subst[i])
            tp[i,j] = tmm.trans_amp(1,ang[j], wave[i], n[:,i], l, 1., n_subst[i])
            ts[i,j] = tmm.trans_amp(0,ang[j], wave[i], n[:,i], l, 1., n_subst[i])
            
    l = the[exp,:]
    n = np.array([materials[mats1[exp]],materials[mats2[exp]]]) #uses the target groud truth material
    rpt = np.zeros((wave.size,ang.size))
    rst = np.zeros((wave.size,ang.size))
    tpt = np.zeros((wave.size,ang.size))
    tst = np.zeros((wave.size,ang.size))
    #calculate the spectra corrsponding to the ground truth structure
    for j in range(0,ang.size):     
        for i in range(0,wave.size):
            rpt[i,j] = tmm.reflect_amp(1,ang[j], wave[i], n[:,i], l, 1., n_subst[i])
            rst[i,j] = tmm.reflect_amp(0,ang[j], wave[i], n[:,i], l, 1., n_subst[i])
            tpt[i,j] = tmm.trans_amp(1,ang[j], wave[i], n[:,i], l, 1., n_subst[i])
            tst[i,j] = tmm.trans_amp(0,ang[j], wave[i], n[:,i], l, 1., n_subst[i])
            
    #spectral RMSE is the root(mean(square(residuals))) between the predicted and input spectra
    rrp = np.sqrt(np.mean(np.mean(np.square(rpt-rp))))
    rrs = np.sqrt(np.mean(np.mean(np.square(rst-rs))))
    rtp = np.sqrt(np.mean(np.mean(np.square(tpt-tp))))
    rts = np.sqrt(np.mean(np.mean(np.square(tst-ts))))
    #average of RMSE for all spectral types
    rmse = np.mean(np.array([rrp,rrs,rtp,rts]))
    
#     #plot every few spectra to get an idea how the model is performing
#     if np.mod(exp,numel/10)==0:
#         plt.plot(wave,rp[:,0],'b--')
#         plt.plot(wave,te_rp[exp,:,0],'b-')
#         plt.plot(wave,rs[:,0],'g--')
#         plt.plot(wave,te_rs[exp,:,0],'g-')
#         plt.show()

    #save the results
    results[exp,0] = rrp
    results[exp,1] = rrs
    results[exp,2] = rtp
    results[exp,3] = rts
    results[exp,4] = rmse
    
#print an average over all of the results
print(np.mean(results,0))

In [ ]:
#ellipsometric to structure model
#timing results

#load the model into memory. Point this to the correct filename
model = mo.load_model('/home/arl92/Documents/newdata/pd2mt/general_2lay4matinverse_model_v-tma-allangle_convolution_RPRSTPTS_1step_20201117')
#calculate the timing results
ti = datetime.datetime.now()
results = model.predict([te_p,te_d])
tf = datetime.datetime.now()
dt = tf-ti

#print the timing results to the output. Returns the average solution time over the enrite test dataset
print('Time Ellapsed:',dt)
print('Time per call:',dt/(te-va))

In [ ]:
#ellipsometric to structure model
#model metrics

#save the predicted results 
th = results[0];
l1 = np.argmax(results[1],1);
l2 = np.argmax(results[2],1);
#evaluate the model loss and metrics (accuracy in materials, thickness mse) against the test dataset
print(model.evaluate([te_p,te_d],[the,m1e,m2e]))

In [ ]:
#ellipsometric to structure model
#spectral RMSE

#import materials
#the wavelengths are copied to coinside with the generated data
wave = np.linspace(450,950,200)*1E-9
ag = bb.nk_material('Ag',wave)
au = bb.nk_material('Au',wave)
al2o3 = di.nk_material('al2o3',wave)
tio2 = di.nk_material('tio2',wave)
ito = di.nk_material('ito',wave)
gl = di.nk_Cauchy_Urbach(wave,1.55,0.005)
void = np.ones(wave.size)
#define the materials array
materials = np.array([ag,al2o3,ito,au,tio2])
ang = np.array([25.,45.,65.])
n_subst = gl #all structures are simulated on a glass substrate

#how many structures to generate spectral RMSE results
numel = 2000 #should be enough to generate a good average
results = np.ones((numel,3))

#decode the materials
mats1 = np.argmax(m1e,1)
mats2 = np.argmax(m2e,1)

for exp in tqdm(range(numel)):
    l = th[exp]*1E-7 #data un-rescaling
    n = np.zeros((l.size,wave.size),dtype=complex)
    p = np.zeros((wave.size,ang.size))
    d = np.zeros((wave.size,ang.size))
    
    n = np.array([materials[l1[exp]],materials[l2[exp]]]) #uses the predicted material
    #calculate the spectra corrsponding to the model predicted structure)
    for j in range(0,ang.size):     
        for i in range(0,wave.size):
            (p[i,j],d[i,j]) = tmm.ellips(ang[j], wave[i], n[:,i], l, 1., n_subst[i])
            
    l = the[exp,:]
    pt = np.zeros((wave.size,ang.size))
    dt = np.zeros((wave.size,ang.size))
    n = np.array([materials[mats1[exp]],materials[mats2[exp]]]) #uses the ground truth material
    for j in range(0,ang.size):     
        for i in range(0,wave.size):
            (pt[i,j],dt[i,j]) = tmm.ellips(ang[j], wave[i], n[:,i], l, 1., n_subst[i])    
    
    #spectral RMSE is the root(mean(square(residuals))) between the predicted and input spectra
    rp = np.sqrt(np.mean(np.mean(np.square(pt-p))))
    rd = np.sqrt(np.mean(np.mean(np.square(dt-d))))
    #average of RMSE for all spectral types
    rmse = np.mean(np.array([rp,rd]))
    
    # save the results to file    
    results[exp,0] = rp
    results[exp,1] = rd
    results[exp,2] = rmse

#print an average over all of the results
print(np.mean(results,0))